## Описание

**Описание:** 
    
    Компания купила крупный сервис для чтения книг по подписке.
    
      Таблица books содержит данные о книгах:
    * book_id — идентификатор книги;
    * author_id — идентификатор автора;
    * title — название книги;
    * num_pages — количество страниц;
    * publication_date — дата публикации книги;
    * publisher_id — идентификатор издателя.
    
    Таблица authors содержит данные об авторах:
    * author_id — идентификатор автора;
    * author — имя автора.
    
    Таблица publishers содержит данные об издательствах:
    * publisher_id — идентификатор издательства;
    * publisher — название издательства;
    
    Таблица ratings содержит данные о пользовательских оценках книг:
    * rating_id — идентификатор оценки;
    * book_id — идентификатор книги;
    * username — имя пользователя, оставившего оценку;
    * rating — оценка книги.
    
    Таблица reviews содержит данные о пользовательских обзорах:
    * review_id — идентификатор обзора;
    * book_id — идентификатор книги;
    * username — имя автора обзора;
    * text — текст обзора.
        
    

**Цель исследования:**


    Анализ базы данных.
    
    
**Ход исследования:**


    1. Описание

    2. Подключение к базе данных
    
    3. Обзор данных
    
    4. Задания в рамках анализа
    
        4.1 Посчитайте, сколько книг вышло после 1 января 2000 года;
        4.2 Для каждой книги посчитайте количество обзоров и среднюю оценку;
        4.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа
        брошюры;
        4.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
        4.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
        
  

## Подключение к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

def connect_and_query_sql(sql_query):
    """
    Параметры:
    sql_query (str): SQL-запрос для выполнения.
    db_type (str): Тип базы данных (например, 'mysql', 'postgresql', 'sqlite').
    username (str): Имя пользователя для подключения.
    password (str): Пароль пользователя.
    host (str): Хост базы данных.
    port (str): Порт для подключения.
    database (str): Имя базы данных.

    """
    db_config = {'user': 'praktikum_student', # имя пользователя
                 'pwd': 'Sdf4$2;d-d30pp', # пароль
                 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
                 'port': 6432, # порт подключения
                 'db': 'data-analyst-final-project-db'} # название базы данных
    
    connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

    engine = create_engine(connection_string, connect_args={'sslmode':'require'})
    
    result_df = pd.read_sql_query(sql_query, engine)
    
    return result_df

## Обзор данных

In [2]:
sql_query = """

SELECT *
FROM books 
WHERE title IS NULL
"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

Empty DataFrame
Columns: [book_id, author_id, title, num_pages, publication_date, publisher_id]
Index: []


In [3]:
sql_query = """

SELECT *
FROM ratings 
LIMIT 5

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

   rating_id  book_id       username  rating
0          1        1     ryanfranco       4
1          2        1  grantpatricia       2
2          3        1   brandtandrea       5
3          4        2       lorichen       3
4          5        2    mariokeller       2


In [4]:
sql_query = """

SELECT *
FROM reviews 
LIMIT 5

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

   review_id  book_id       username  \
0          1        1   brandtandrea   
1          2        1     ryanfranco   
2          3        2       lorichen   
3          4        3  johnsonamanda   
4          5        3    scotttamara   

                                                text  
0  Mention society tell send professor analysis. ...  
1  Foot glass pretty audience hit themselves. Amo...  
2  Listen treat keep worry. Miss husband tax but ...  
3  Finally month interesting blue could nature cu...  
4  Nation purpose heavy give wait song will. List...  


In [5]:
sql_query = """

SELECT *
FROM authors 
LIMIT 5

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Lloyd


In [6]:
sql_query = """

SELECT *
FROM publishers 
LIMIT 5

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

   publisher_id                          publisher
0             1                                Ace
1             2                           Ace Book
2             3                          Ace Books
3             4                      Ace Hardcover
4             5  Addison Wesley Publishing Company


## Задания в рамках анализа

### Количество книг, вышедших после 1 января 2000 года

In [7]:
sql_query = """

SELECT COUNT(book_id) 
FROM books 
WHERE publication_date > '2000-01-01'

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

   count
0    819


После 1 января 2000 года вышло 819 книг

### Количество обзоров и средняя оценка для каждой книги

In [8]:
sql_query = """

WITH average_mark AS 
(SELECT b.book_id, AVG(rt.rating) AS a
FROM books AS b
FULL JOIN ratings as rt ON b.book_id=rt.book_id
GROUP BY b.book_id),

count_reviews AS
(SELECT b.book_id, title, COUNT(review_id) AS c
FROM books AS b
FULL JOIN reviews as rw ON b.book_id=rw.book_id
GROUP BY b.book_id)

SELECT title, c, a 
FROM average_mark 
FULL JOIN count_reviews ON average_mark.book_id=count_reviews.book_id
ORDER BY c DESC, a DESC
"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

                                                 title  c         a
0                              Twilight (Twilight  #1)  7  3.662500
1    Harry Potter and the Prisoner of Azkaban (Harr...  6  4.414634
2    Harry Potter and the Chamber of Secrets (Harry...  6  4.287500
3                                       The Book Thief  6  4.264151
4                                     The Glass Castle  6  4.206897
..                                                 ... ..       ...
995  Disney's Beauty and the Beast (A Little Golden...  0  4.000000
996                               Leonardo's Notebooks  0  4.000000
997                          Essential Tales and Poems  0  4.000000
998    Anne Rice's The Vampire Lestat: A Graphic Novel  0  3.666667
999                            The Natural Way to Draw  0  3.000000

[1000 rows x 3 columns]


Самые обсуждаемые книги (судя по количеству обзоров) далеко не всегда обладают высоким рейтингом и наоборот.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
sql_query = """

WITH cp AS 

(SELECT publisher, COUNT(DISTINCT b.book_id) AS c 
FROM books AS b 
JOIN publishers AS p ON b.publisher_id=p.publisher_id 
WHERE num_pages > 50 
GROUP BY p.publisher_id
ORDER BY c DESC) 

SELECT publisher, c
FROM cp 
WHERE c = (SELECT MAX(c) FROM cp)

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

       publisher   c
0  Penguin Books  42


Самое продуктивное, с точки зрения количества выпущенных книг, издательство - Penguin Books (42 книги)

### Автор с самой высокой средней оценкой книг с 50 и более оценками

In [10]:
sql_query = """

WITH filt_rate AS

    (WITH count_rate AS 

    (SELECT b.book_id, COUNT(rating_id) AS c
    FROM books AS b 
    JOIN ratings AS r ON b.book_id=r.book_id 
    GROUP BY b.book_id
    ORDER BY COUNT(rating_id))

SELECT book_id, c
FROM count_rate
WHERE c > 50),

filt_auth AS 

(SELECT author, AVG(rating) AS av
FROM books AS b
INNER JOIN filt_rate ON filt_rate.book_id=b.book_id
JOIN ratings AS r ON b.book_id=r.book_id 
JOIN authors AS a ON a.author_id=b.author_id
GROUP BY author)

SELECT author
FROM filt_auth
WHERE av = (SELECT MAX(av) FROM filt_auth)

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

                       author
0  J.K. Rowling/Mary GrandPré


У книг Роулинг самая высокая средняя оценка (по всему миру огромное количество людей любят серию книг про Гарри).

### Среднее количество обзоров от пользователей, поставивших больше 48 оценок

In [11]:
sql_query = """

WITH users_marks AS
(SELECT username, COUNT(rating_id) AS c
FROM ratings 
GROUP BY username),

users_top_marks AS
(SELECT username
FROM users_marks
WHERE c > 48),

users_top_reviews AS 
(SELECT COUNT(review_id) AS cr
FROM users_top_marks
INNER JOIN reviews AS r ON users_top_marks.username = r.username
GROUP BY r.username)

SELECT AVG(cr)
FROM users_top_reviews

"""

result_dataframe = connect_and_query_sql(sql_query)
print(result_dataframe)

    avg
0  24.0


Активные пользователи, как минимум в 2 раза чаще просто ставят оценки, чем пишут отзывы.

## Общий вывод

В ходе исследования мы обнаружили, что самое продуктивное издательство - Penguin Books, самая успешная писательница - Джоан Роулинг, а также что в 21 веке на платформе появилось 819 книг, рейтинг книг напрямую не коррелирует с количество отзывов и на каждый обзор у активных пользователей приходится как минимум 2 оценки.